# Experimento de performance e Avaliação de Consumo do Fabric com o Duckdb em sessão Python

Com este experimento simples, estou usando dados de diesel em dois cenários:
- Série histórica de preços de 4 anos totalizando mais de 700mb em csv;
- Arquivo simples de menos de 1mb com informações geofísicas;

A transformação significava trazer de csv consolidado para parquet com duckdb e fazer umas transformações simples.

Sessão python 3.11

In [30]:
# Código DuckDB em cenário de consolidação dos dados históricos e pequenas transformações (>700MB)
import time
import psutil
import duckdb
from contextlib import contextmanager

class DetailedMetrics:
    def __init__(self):
        self.measures = {}
        
    @contextmanager
    def measure_step(self, step_name):
        start = time.time()
        start_mem = psutil.Process().memory_info().rss
        start_io = psutil.disk_io_counters()
        
        try:
            yield
        finally:
            end = time.time()
            end_mem = psutil.Process().memory_info().rss
            end_io = psutil.disk_io_counters()
            
            self.measures[step_name] = {
                'time': end - start,
                'memory': (end_mem - start_mem) / 1024 / 1024,
                'io_read': end_io.read_bytes - start_io.read_bytes,
                'io_write': end_io.write_bytes - start_io.write_bytes
            }
            print(f"\nMétricas para {step_name}:")
            print(f"Tempo: {self.measures[step_name]['time']:.2f}s")
            print(f"Memória: {self.measures[step_name]['memory']:.2f} MB")
            print(f"IO Leitura: {self.measures[step_name]['io_read'] / 1024 / 1024:.2f} MB")
            print(f"IO Escrita: {self.measures[step_name]['io_write'] / 1024 / 1024:.2f} MB")

metrics = DetailedMetrics()
con = duckdb.connect()

# Leitura
with metrics.measure_step("DuckDB - Leitura"):
    con.execute("""
    CREATE TABLE df_concatenado AS
    SELECT * FROM read_csv_auto('/lakehouse/default/Files/diesel/serie_historica_csv/historico_diesel_2023_01.csv', delim=';', header=True)
    UNION ALL
    SELECT * FROM read_csv_auto('/lakehouse/default/Files/diesel/serie_historica_csv/historico_diesel_2023_02.csv', delim=';', header=True)
    UNION ALL
    SELECT * FROM read_csv_auto('/lakehouse/default/Files/diesel/serie_historica_csv/historico_diesel_2024_01.csv', delim=';', header=True)
    UNION ALL
    SELECT * FROM read_csv_auto('/lakehouse/default/Files/diesel/serie_historica_csv/historico_diesel_2024_02.csv', delim=';', header=True)
    UNION ALL
    SELECT * FROM read_csv_auto('/lakehouse/default/Files/diesel/serie_historica_csv/historico_diesel_2021_01.csv', delim=';', header=True)
    UNION ALL
    SELECT * FROM read_csv_auto('/lakehouse/default/Files/diesel/serie_historica_csv/historico_diesel_2022_01.csv', delim=';', header=True)
    UNION ALL
    SELECT * FROM read_csv_auto('/lakehouse/default/Files/diesel/serie_historica_csv/historico_diesel_2022_02.csv', delim=';', header=True)
    """)

# Transformação
with metrics.measure_step("DuckDB - Transformação"):
    con.execute("""
    CREATE TABLE df_processado AS
    SELECT DISTINCT
        "Produto",
        "Data da Coleta",
        "Regiao - Sigla" as "Regiao",
        "Estado - Sigla" as "Estado",
        "Municipio",
        COALESCE("Valor de Compra", '0') as "Valor de Compra",
        COALESCE("Complemento", 'SEM COMPLEMENTO') as "Complemento",
        CAST(REPLACE("Valor de Venda", ',', '.') AS DOUBLE) as "Valor de Venda",
        CAST(REPLACE(COALESCE("Valor de Compra", '0'), ',', '.') AS DOUBLE) as "Valor de Compra_Num",
        CAST(REPLACE("Valor de Venda", ',', '.') AS DOUBLE) - 
        CAST(REPLACE(COALESCE("Valor de Compra", '0'), ',', '.') AS DOUBLE) as "Margem"
    FROM df_concatenado
    """)

# Escrita
with metrics.measure_step("DuckDB - Escrita"):
    con.execute("COPY df_processado TO '/lakehouse/default/Files/diesel/serie_historica_parquet/diesel_4anos_duckdb.parquet' (FORMAT 'parquet')")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Métricas para DuckDB - Leitura:
Tempo: 7.38s
Memória: 830.58 MB
IO Leitura: 0.59 MB
IO Escrita: 12.36 MB

Métricas para DuckDB - Transformação:
Tempo: 1.73s
Memória: 207.29 MB
IO Leitura: 0.30 MB
IO Escrita: 0.12 MB


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Métricas para DuckDB - Escrita:
Tempo: 2.37s
Memória: 145.16 MB
IO Leitura: 0.05 MB
IO Escrita: 2.09 MB


In [29]:
#Código duckdb com o cenário de arquivo pequeno (<1MB)
import duckdb
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import time
import psutil
import os

def get_memory_usage():
   process = psutil.Process(os.getpid())
   return process.memory_info().rss / 1024 / 1024  # MB

# Início do benchmark
start_time = time.time()
initial_memory = get_memory_usage()

con = duckdb.connect()
query = """
SELECT 
   Nome as id_programa,
   Categoria,
   CASE WHEN Natureza = 'Não-Exclusivo' THEN 'NAO_EXCLUSIVO' ELSE 'EXCLUSIVO' END as tipo_natureza,
   TRY_CAST(Inicio AS DATE) as data_inicio,
   TRY_CAST("Término Real" AS DATE) as data_termino,
   Tecnologia,
   Bacia,
   COALESCE(Operadora, 'NAO_INFORMADO') as operadora,
   DATEDIFF('month', TRY_CAST(Inicio AS DATE), TRY_CAST("Término Real" AS DATE)) as duracao_meses,
   CASE WHEN Bacia = 'Santos' THEN 'BACIA_SANTOS' WHEN Bacia = 'Campos' THEN 'BACIA_CAMPOS' ELSE 'OUTRAS' END as categoria_bacia,
   CASE WHEN Tecnologia = 'Sísmica 3D' THEN 'SISMICA_3D' WHEN Tecnologia = 'Magnetometria' THEN 'MAGNETOMETRIA' 
        WHEN Tecnologia = 'Ocean Bottom Nodes' THEN 'OBN' ELSE 'OUTROS' END as tipo_tecnologia
FROM read_csv_auto('/lakehouse/default/Files/diesel/pequenos/tabela-programas-geofisicos.csv', sep=';', skip=1)
"""

df = con.execute(query).df()
query_time = time.time() - start_time

# Conversão e gravação
table = pa.Table.from_pandas(df)
pq.write_table(table, '/lakehouse/default/Files/diesel/pequenos/programas_geofisicos_benchmark.parquet')

total_time = time.time() - start_time
peak_memory = get_memory_usage() - initial_memory

print(f"Métricas de Performance:")
print(f"Tempo total: {total_time:.2f} segundos")
print(f"Tempo query: {query_time:.2f} segundos")
print(f"Memória utilizada: {peak_memory:.2f} MB")
print(f"Linhas processadas: {len(df)}")

con.close()

Métricas de Performance:
Tempo total: 0.11 segundos
Tempo query: 0.03 segundos
Memória utilizada: 1.50 MB
Linhas processadas: 3604
